In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import time
from keras.applications import vgg16
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential,load_model
from keras.utils import to_categorical
import keras
from keras.layers import InputLayer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
data_path = '/home/vishwajeet/Documents/Summer_Project_FER/CK+_model/CK+48/'
data_dir_list = os.listdir(data_path)

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.resize(input_img,(96,96))
        img_data_list.append(input_img)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-happy

Loaded the images of dataset-anger

Loaded the images of dataset-surprise

Loaded the images of dataset-sadness

Loaded the images of dataset-disgust

Loaded the images of dataset-fear



(927, 96, 96, 3)

In [3]:
num_classes = 6

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:135]=0 #135
labels[135:312]=1 #177
labels[312:387]=2 #75
labels[387:594]=3 #207
labels[594:678]=4 #84
labels[678:927]=5 #249

# labels =to_categorical(labels, num_classes)
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [4]:
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(img_data, labels)

10

In [5]:
print(skf)
vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(96, 96, 3)))

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
input_shape = vgg_model.output_shape[1]

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [6]:
def cnn_model(x_train,x_test,y_test,y_train):
    model = Sequential()
    model.add(vgg_model)
    model.add(Dense(512, activation='relu', input_dim=input_shape))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(6, activation='sigmoid'))

    model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 


    model.fit(x_train, y_train,epochs=35, batch_size=80,verbose=2)
    model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics=['accuracy'],)

    loss,acc = model.evaluate(x_test, y_test)
    y_prediction = model.predict_classes(x_test)
    y_test_original=np.argmax(y_test,axis=1)
    confusion=confusion_matrix(y_true=y_test_original, y_pred=y_prediction)
    return acc,confusion

In [7]:
prediction=[]
confusion_list=[]
IMAGE_WIDTH=96
IMAGE_HEIGHT=96
num_classes = 6
for train_index, test_index in skf.split(img_data, labels):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = img_data[train_index], img_data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    x_train=x_train.reshape(x_train.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
    x_test=x_test.reshape(x_test.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
    y_train =to_categorical(y_train, num_classes)
    y_test =to_categorical(y_test, num_classes)
    X,Y=cnn_model(x_train,x_test,y_test,y_train)
    prediction.append(X)
    confusion_list.append(Y)
    print(X)
    print(Y)
    print("----------------------------")

Epoch 1/35
 - 70s - loss: 1.5366 - accuracy: 0.3978
Epoch 2/35
 - 64s - loss: 1.1774 - accuracy: 0.5312
Epoch 3/35
 - 68s - loss: 0.9641 - accuracy: 0.6322
Epoch 4/35
 - 64s - loss: 0.7595 - accuracy: 0.7175
Epoch 5/35
 - 68s - loss: 0.6264 - accuracy: 0.7548
Epoch 6/35
 - 67s - loss: 0.5143 - accuracy: 0.7921
Epoch 7/35
 - 64s - loss: 0.4900 - accuracy: 0.8137
Epoch 8/35
 - 66s - loss: 0.4162 - accuracy: 0.8269
Epoch 9/35
 - 64s - loss: 0.3952 - accuracy: 0.8305
Epoch 10/35
 - 67s - loss: 0.3826 - accuracy: 0.8305
Epoch 11/35
 - 64s - loss: 0.3316 - accuracy: 0.8450
Epoch 12/35
 - 66s - loss: 0.3098 - accuracy: 0.8438
Epoch 13/35
 - 67s - loss: 0.3372 - accuracy: 0.8582
Epoch 14/35
 - 64s - loss: 0.3276 - accuracy: 0.8558
Epoch 15/35
 - 68s - loss: 0.2950 - accuracy: 0.8570
Epoch 16/35
 - 63s - loss: 0.2854 - accuracy: 0.8666
Epoch 17/35
 - 68s - loss: 0.2389 - accuracy: 0.8690
Epoch 18/35
 - 67s - loss: 0.2634 - accuracy: 0.8510
Epoch 19/35
 - 67s - loss: 0.2583 - accuracy: 0.8618
Ep

Epoch 1/35
 - 70s - loss: 0.9599 - accuracy: 0.6675
Epoch 2/35
 - 66s - loss: 0.3567 - accuracy: 0.8451
Epoch 3/35
 - 63s - loss: 0.2744 - accuracy: 0.8523
Epoch 4/35
 - 64s - loss: 0.2708 - accuracy: 0.8571
Epoch 5/35
 - 65s - loss: 0.2466 - accuracy: 0.8619
Epoch 6/35
 - 64s - loss: 0.2537 - accuracy: 0.8691
Epoch 7/35
 - 65s - loss: 0.2452 - accuracy: 0.8788
Epoch 8/35
 - 64s - loss: 0.2425 - accuracy: 0.8595
Epoch 9/35
 - 64s - loss: 0.2381 - accuracy: 0.8703
Epoch 10/35
 - 65s - loss: 0.2462 - accuracy: 0.8571
Epoch 11/35
 - 64s - loss: 0.2361 - accuracy: 0.8535
Epoch 12/35
 - 64s - loss: 0.2346 - accuracy: 0.8667
Epoch 13/35
 - 64s - loss: 0.2443 - accuracy: 0.8752
Epoch 14/35
 - 64s - loss: 0.2282 - accuracy: 0.8848
Epoch 15/35
 - 65s - loss: 0.2457 - accuracy: 0.8739
Epoch 16/35
 - 61s - loss: 0.2208 - accuracy: 0.8944
Epoch 17/35
 - 64s - loss: 0.2288 - accuracy: 0.8739
Epoch 18/35
 - 64s - loss: 0.2270 - accuracy: 0.8824
Epoch 19/35
 - 65s - loss: 0.2244 - accuracy: 0.8703
Ep

Epoch 1/35
 - 75s - loss: 0.8134 - accuracy: 0.7097
Epoch 2/35
 - 72s - loss: 0.2433 - accuracy: 0.8757
Epoch 3/35
 - 72s - loss: 0.2076 - accuracy: 0.9056
Epoch 4/35
 - 72s - loss: 0.1885 - accuracy: 0.9080
Epoch 5/35
 - 77s - loss: 0.1815 - accuracy: 0.9104
Epoch 6/35
 - 75s - loss: 0.1847 - accuracy: 0.9140
Epoch 7/35
 - 72s - loss: 0.1891 - accuracy: 0.8984
Epoch 8/35
 - 73s - loss: 0.1862 - accuracy: 0.9104
Epoch 9/35
 - 71s - loss: 0.1933 - accuracy: 0.8996
Epoch 10/35
 - 70s - loss: 0.1695 - accuracy: 0.9188
Epoch 11/35
 - 76s - loss: 0.1818 - accuracy: 0.9128
Epoch 12/35
 - 71s - loss: 0.1851 - accuracy: 0.9008
Epoch 13/35
 - 73s - loss: 0.1713 - accuracy: 0.9056
Epoch 14/35
 - 72s - loss: 0.1701 - accuracy: 0.9104
Epoch 15/35
 - 71s - loss: 0.1927 - accuracy: 0.9020
Epoch 16/35
 - 68s - loss: 0.1722 - accuracy: 0.9176
Epoch 17/35
 - 72s - loss: 0.1666 - accuracy: 0.9104
Epoch 18/35
 - 68s - loss: 0.1556 - accuracy: 0.9247
Epoch 19/35
 - 69s - loss: 0.1553 - accuracy: 0.9271
Ep

In [8]:
print(prediction)

[0.6736842393875122, 0.6842105388641357, 0.800000011920929, 0.7368420958518982, 0.8936170339584351, 0.8152173757553101, 0.8260869383811951, 0.855555534362793, 0.8777777552604675, 0.8089887499809265]


In [13]:
# print(confusion_list)
for x in range(len(confusion_list)): 
    print (confusion_list[x],sep = "\n")
    print("----------------------------")    

[[ 5  9  0  0  0  0]
 [ 8 10  0  0  0  0]
 [ 0  8  0  0  0  0]
 [ 0  0  4 17  0  0]
 [ 0  0  0  0  9  0]
 [ 0  0  0  0  2 23]]
----------------------------
[[13  1  0  0  0  0]
 [14  4  0  0  0  0]
 [ 0  4  4  0  0  0]
 [ 0  0 10 11  0  0]
 [ 0  0  0  0  9  0]
 [ 0  0  0  0  1 24]]
----------------------------
[[12  2  0  0  0  0]
 [ 8 10  0  0  0  0]
 [ 0  6  2  0  0  0]
 [ 0  0  3 18  0  0]
 [ 0  0  0  0  9  0]
 [ 0  0  0  0  0 25]]
----------------------------
[[10  4  0  0  0  0]
 [13  5  0  0  0  0]
 [ 0  5  3  0  0  0]
 [ 0  0  2 19  0  0]
 [ 0  0  0  0  9  0]
 [ 0  0  0  0  1 24]]
----------------------------
[[14  0  0  0  0  0]
 [ 0 17  1  0  0  0]
 [ 0  0  2  6  0  0]
 [ 0  0  3 18  0  0]
 [ 0  0  0  0  8  0]
 [ 0  0  0  0  0 25]]
----------------------------
[[11  2  0  0  0  0]
 [ 0 11  7  0  0  0]
 [ 0  0  1  6  0  0]
 [ 0  0  1 20  0  0]
 [ 0  0  0  1  7  0]
 [ 0  0  0  0  0 25]]
----------------------------
[[ 7  6  0  0  0  0]
 [ 0 15  3  0  0  0]
 [ 0  0  3  4  0  0]
 